# Class to calculate the total fluorescence intensity of area covered by nucleus in raw fluorescence channel image

In [13]:
import os
import sys
import h5py
import math
import numpy as np
import scipy.spatial as sp
import matplotlib.pyplot as plt

sys.path.append("../")

from scipy.ndimage import label, find_objects
from tqdm import tqdm
from PIL import Image
from skimage import io
from Movie_Analysis_Pipeline.Single_Movie_Processing.Server_Movies_Paths import Get_MDCK_Movies_Paths

In [28]:
class Fluo_Signal_Intensity(object):

    def __init__(self, hdf5_file):
        """ Open & read data from chosen HDF5 file. TODO: This class only processes GFP cells. Deal with it!
            :param hdf5_file (str):                 absolute directory to file: .../HDF/segmented.hdf5
        """

        self.hdf5_file = hdf5_file
        self.hdf5_file_to_read = h5py.File(hdf5_file, 'r')
        self.movie_length = len(self.hdf5_file_to_read["objects"]["obj_type_1"]["map"])
        self.channels = len(list(self.hdf5_file_to_read.values())[0])
        
        GFP_length = len(self.hdf5_file_to_read["objects"]["obj_type_1"]["coords"])

        if "obj_type_1" not in list(self.hdf5_file_to_read["objects"]):
            raise ValueError("GFP channel not detected in the HDF5 file.")

        self.position = hdf5_file.split("/pos")[1].split("/")[0]
        self.data_date = hdf5_file.split("/pos{}".format(self.position))[0][-6:]
        
        self.raw_movie = "/Volumes/lowegrp/Data/Kristina/MDCK_WT_Pure/17_{}_{}/pos{}/GFP_pos8.tif" \
                            .format(self.data_date[2:4], self.data_date[4:6], self.position)
        
        self.raw_image_sequence = io.imread(movie_GFP)
        
        if not self.data_date.startswith("AB") and not self.data_date.startswith("GV"):
            raise AttributeError("Wrong HDF file initiation on position <{}> or data_date <{}>.".format(self.position, self.data_date))

        # Vectors to return:
        self.nucleus = [0 for _ in range(GFP_length)]
        self.fsignal = [0 for _ in range(GFP_length)]


    def Extract_Cell_Coords(self, frame):
        """ Extract the GFP and RFP cell coordinates, remembering the indexes of these cells.

        :param      frame           (int)
        :return:    cell_coords     (numpy.ndarray)     [[x_coord, y_coord] [x_coord, y_coord] ... ]
                    cell_map        (numpy.ndarray)     [[0 88] [0 20]] -> indices of GFP & RFP cells per frame
        """

        cell_coords = []
        cell_map = []
        
        for channel in range(1, self.channels + 1):
            map = self.hdf5_file_to_read["objects"]["obj_type_{}".format(channel)]["map"][frame]
            cell_map.append(map)
            for cell in range(map[0], map[1]):
                cell_data = self.hdf5_file_to_read["objects"]["obj_type_{}".format(channel)]["coords"][cell]
                cell_coords.append([cell_data[1], cell_data[2]])
        
        return np.array(cell_coords), np.array(cell_map)


    def Calculate_Nuclei_Sizes(self, frame, show=False):
        """ Process the respective binary mask (U-Net output with segmented labels)
            to extract the pixel values of the image into 2D matrix to return.

            1.) Import the 'segmentation' binary mask image & label the pixel values of individual objects.
            2.) Allocate the nuclei centroids from HDF file to each uniquely labelled blob in the binary mask.
            3.) Count the occurence of the label in the processed binary mask & store it's row & column indices.
            4.) Access the corresponding pixels in the raw fluorescence image to calculate average signal intensity.
        """

        cell_coords, cell_map = self.Extract_Cell_Coords(frame=frame)
        pixels = self.hdf5_file_to_read["segmentation"]["images"][frame]

        # Enumerate different objects in the map with unique label & find those objects in the image:
        object_labels, num_features = label(input=pixels)
        found_objects = find_objects(object_labels)

        if num_features != len(found_objects):
            raise ValueError("Warning, number of labelled objects & the objects found with unique label are not equal!")

        # Visualise the binary map & labelled map:
        if show is True:
            plt.imshow(X=pixels)  # plots a 2D array straight ahead!
            plt.title("Raw Segmented Binary Mask at frame #{}".format(frame))
            plt.show()
            plt.close()

            plt.imshow(X=object_labels)  # plots a 2D array straight ahead!
            plt.title("Labelled Segmented Binary Mask at frame #{}".format(frame))
            plt.show()
            plt.close()

        # Match coords to its unique label & sum the appearance of the label in the slice:
        nuclei_size = []
        for coords in cell_coords:
            x, y = int(math.floor(coords[0])), int(math.floor(coords[1]))
            pixel_label = object_labels[x][y]
            image_slice = object_labels[found_objects[pixel_label - 1]]
            nucleus_size = sum([list(row).count(pixel_label) for row in image_slice])
            nuclei_size.append(nucleus_size)

        # Append these sizes into the final array:
        self.nucleus[cell_map[0][0]:cell_map[0][1]] = np.array(nuclei_size, dtype=np.uint8)

        #print (object_labels)
        #print (found_objects)
        
        return nuclei_size, object_labels, found_objects


    # ---------------------------------------------------------------------------------------------------

    def Calculate_Fluo_Intensity(self, frame, show=False):
        """ Calculate the average fluorescence intensity of the nucleus based on the pixel value readouts
            from areas superimposed by uniquely labelled binary mask areas by summing them up & averaging.

            :param! raw_images  (str)   ->      absolute directory to folder:

                    Anna's movies:      e.g./Volumes/lowegrp/Data/Kristina/MDCK_WT_Pure/17_07_31/pos8/...
                                            which contains STACK TIFFs: 'BF_pos8.tif', 'GFP_pos8.tif', 'RFP_pos8.tif'
                    Giulia's movies:    e.g./Volumes/lowegrp/Data/Guilia/GV0800/pos0/Pos0_aligned/
                                            which contains original BF, GFP & RFP images: e.g.
                                            'img_channel001_position013_time000001104_z000.tif'
        """

        cell_coords, cell_map = self.Extract_Cell_Coords(frame=frame)
        nuclei_size, object_labels, found_objects = self.Calculate_Nuclei_Sizes(frame=frame)
        fluo_signal_int = [0 for _ in range(len(nuclei_size))]

        if self.data_date.startswith("AB"):
            img = self.raw_image_sequence[frame]
            print ("Type: {}".format(type(img)))
            print ("Image: {}".format(img))
            
        
        elif self.data_date.startswith("GV"):
            img = "/Volumes/lowegrp/Data/Giulia/{}/pos{}/Pos{}_aligned/img_channel001_position{}_time00000{}_z000.tif"\
                   .format(self.data_date, self.position, self.position, self.position.zfill(3), str(frame).zfill(4))
            print ("Image: {}".format(img))
        
            if not os.path.isfile(img):
                return np.array(fluo_signal_int, dtype=np.float64)

            # Process the full-sized image:
            image = Image.open(img).convert('L')          # converts the image to 8-bit grayscale
            img_w, img_h = image.size                     # stores image dimensions

            # Define center & crop image accordingly:
            new_w, new_h = 1600, 1200
            if img_w != new_w or img_h != new_h:
                left = (img_w - new_w) / 2
                top = (img_h - new_h) / 2
                right = (img_w + new_w) / 2
                bottom = (img_h + new_h) / 2
                image = image.crop((left, top, right, bottom))

            data = np.array(image.getdata())              # converts data to single pixel list; len = 1600 x 1200
            pixels = [data[offset:offset + img_w] for offset in range(0, img_w * img_h, img_w)]
                # converts data to 2D list (list of 'numpy.ndarray' of 'numpy.int64'); access via pixels[row][col]

            # Visualise the image:
            if show is True:
                plt.imshow(X=pixels)
                plt.title("GFP image ({} x {} pixels) at frame #{}".format(img_w, img_h, frame))
                plt.show()
                plt.close()

        # Check whether the dimensions of your uniquely labelled image & your raw fluo image are the same:
        fluo_raw_im = np.array(img)     # convert 'PIL.Image.Image' to 'numpy.ndarray'

        if object_labels.shape[0] != fluo_raw_im.shape[0] or object_labels.shape[1] != fluo_raw_im.shape[1]:
            raise ValueError("Dimensions of uniquely labelled image <{}> & raw fluorescence image <{}> "
                             "are not matching! -> It should be <{}>".format(object_labels.shape,
                                                             fluo_raw_im.shape, (new_h, new_w)))

        # Superimpose the segmented masks with unique labels to the raw fluorescence readout images:
        if len(cell_coords) != len(nuclei_size):
            raise ValueError("Not every cell nucleus had had it's size calculated.")

        for enum, (coords, size) in enumerate(zip(cell_coords, nuclei_size)):

            if size == 0:
                fluo_signal_int[enum] = 0.0

            else:
                x, y = int(math.floor(coords[0])), int(math.floor(coords[1]))
                pixel_label = object_labels[x][y]
                found_loc = found_objects[pixel_label - 1]
                image_slice_mask = object_labels[found_loc]
                image_slice_fluo = fluo_raw_im[found_loc]

                fluo_value_sum = 0
                for row_mask, row_fluo in zip(image_slice_mask, image_slice_fluo):
                    for label_pixel, raw_pixel in zip(row_mask, row_fluo):
                        if label_pixel == pixel_label:
                            fluo_value_sum += raw_pixel

                fluo_signal_int[enum] = float(float(fluo_value_sum) / float(size))

        self.fsignal[cell_map[0][0]:cell_map[0][1]] = np.array(fluo_signal_int, dtype=np.float64)
        return np.array(fluo_signal_int, dtype=np.float64)
            
            
    # ---------------------------------------------------------------------------------------------------

    def Process_Whole_Movie(self):
        """ """

        #for frame in tqdm(range(0, self.movie_length)):
        for frame in tqdm(range(0, 10)):

            if frame % 100 == 0:
                print("\nCalculating for frame #{} out of {} frames...".format(frame, self.movie_length))

            self.Calculate_Fluo_Intensity(frame=frame)

        if self.hdf5_file_to_read.__bool__():
            self.hdf5_file_to_read.close()

        return self.fsignal


    """
    def Append_To_HDF(self, local_density=False, nucleus_size=False, fluo_signal=False):

        density, nucleus, fsignal = self.Process_Whole_Movie(local_density=local_density,
                                                             nucleus_size=nucleus_size,
                                                             fluo_signal=fluo_signal)

        with h5py.File(self.hdf5_file, 'a') as f:
            if "density" in list(f["objects"]["obj_type_1"]):
                del f["objects"]["obj_type_1"]["density"]

            if "local_density" in list(f["objects"]["obj_type_1"]):
                del f["objects"]["obj_type_1"]["local_density"]

            grp_d = f["objects"]["obj_type_1"]
            grp_d.create_dataset(name="local_density", data=density)

            if "nucleus_size" in list(f["objects"]["obj_type_1"]):
                del f["objects"]["obj_type_1"]["nucleus_size"]

            grp_n = f["objects"]["obj_type_1"]
            grp_n.create_dataset(name="nucleus_size", data=nucleus)
    """
    
    def __exit__(self):
        self.hdf5_file_to_read.close()

In [30]:
# Call the class:

hdf5_file = "/Volumes/lowegrp/Data/Kristina/Cells_MDCK/AB0124/pos7/HDF/segmented.hdf5"
call = Fluo_Signal_Intensity(hdf5_file=hdf5_file).Process_Whole_Movie()

print (len(call), call[0:100])

"""
movies = Get_MDCK_Movies_Paths()

for movie in movies:
    hdf5_file = movie + "HDF/segmented.hdf5"
    print ("Calculating for {}".format(hdf5_file))
    Local_Density_Nucleus_Size_Fluo_Signal(hdf5_file=hdf5_file).Append_To_HDF(local_density=True, nucleus_size=True)
"""

  0%|          | 0/10 [00:00<?, ?it/s]


Calculating for frame #0 out of 1793 frames...


 10%|█         | 1/10 [00:00<00:03,  2.63it/s]

Type: <class 'numpy.ndarray'>
Image: [[224 150   8 ...   4   3   2]
 [  6   6   6 ...   3   3   4]
 [  5   7   6 ...   3   3   2]
 ...
 [  4   4   5 ...   3   2   3]
 [  4   5   6 ...   3   3   4]
 [  5   5   4 ...   4   2   3]]


 20%|██        | 2/10 [00:00<00:03,  2.66it/s]

...

 90%|█████████ | 9/10 [00:03<00:00,  2.74it/s]

Type: <class 'numpy.ndarray'>
Image: [[ 77 146  48 ...   3   3   2]
 [  7   7   6 ...   3   4   3]
 [  5   6   6 ...   4   3   2]
 ...
 [  4   5   4 ...   4   2   3]
 [  4   4   5 ...   2   2   2]
 [  5   4   5 ...   4   3   2]]


100%|██████████| 10/10 [00:03<00:00,  2.73it/s]

Type: <class 'numpy.ndarray'>
Image: [[251  39 171 ...   3   4   4]
 [  6   7   6 ...   3   3   2]
 [  5   6   5 ...   3   4   3]
 ...
 [  5   5   3 ...   2   3   2]
 [  5   4   6 ...   2   4   3]
 [  4   4   6 ...   2   3   4]]
715350 [6.623529411764705, 6.765625, 6.413861386138614, 7.208163265306123, 5.961728395061728, 5.125, 4.092307692307692, 5.191512513601741, 3.0878274268104775, 6.564154786150713, 6.244660194174758, 4.758695652173913, 5.317357512953368, 4.963636363636364, 5.827160493827161, 6.1210191082802545, 6.338983050847458, 5.403934426229508, 5.3328125, 6.080118694362018, 6.272588055130169, 5.906600249066003, 5.421530479896239, 4.736916548797737, 5.914407988587731, 5.3674911660777385, 5.571428571428571, 6.177914110429448, 6.132352941176471, 5.486005089058525, 5.763341067285383, 5.254830917874396, 5.508528784648187, 8.963414634146341, 5.969230769230769, 5.64248159831756, 5.209705372616984, 6.1122278056951425, 4.744154057771665, 6.025445292620865, 5.773858921161826, 5.26718213

'\nmovies = Get_MDCK_Movies_Paths()\n\nfor movie in movies:\n    hdf5_file = movie + "HDF/segmented.hdf5"\n    print ("Calculating for {}".format(hdf5_file))\n    Local_Density_Nucleus_Size_Fluo_Signal(hdf5_file=hdf5_file).Append_To_HDF(local_density=True, nucleus_size=True)\n'